In [3]:
import math
import numpy as np

In [4]:
def printPoints(vectors):
    formatted_vectors = []
    for vector in vectors:
        formatted_vector = []
        for value in vector:
            if value == int(value):
                formatted_vector.append(f"{int(value)}")
            else:
                formatted_vector.append(f"{value:.2f}")
        formatted_vectors.append(f"({', '.join(formatted_vector)})")

    print('Copy and paste the following directly into Desmos:')
    for ind, vector in enumerate(formatted_vectors):
      print(f'O_{ind} = {vector}')

In [62]:
def generate_points_on_plane(center, radius, point_a, point_b, num_points):
    # Convert points to numpy arrays
    center = np.array(center)
    point_a = np.array(point_a)
    point_b = np.array(point_b)

    # Calculate vectors CA and CB
    vec_ca = point_a - center
    vec_cb = point_b - center

    # Normal vector of the plane (cross product of CA and CB)
    normal = np.cross(vec_ca, vec_cb)

    # Find one basis vector on the plane (cross product of normal and CA)
    u = vec_ca / np.linalg.norm(vec_ca)  # Normalize

    # Find another basis vector on the plane (cross product of normal and U)
    v = np.cross(normal, u)
    v = v / np.linalg.norm(v)  # Normalize

    # Circle's radius
    circle_radius = 2 * radius

    # Generate points on the circle
    points = []
    for i in range(num_points):
        theta = 2 * np.pi * i / num_points
        point = center + circle_radius * (np.cos(theta) * u + np.sin(theta) * v)
        points.append(np.round(point, 2))

    return np.array(points)


In [63]:
def createPointOnSphere(center, radius, theta, phi):
  h, k, l = center[0], center[1], center[2]
  x_1 = h + radius * math.sin(theta) * math.cos(phi)
  y_1 = k + radius * math.sin(theta) * math.sin(phi)
  z_1 = l + radius * math.cos(theta)
  return (x_1, y_1, z_1)

In [90]:
center = (0,0,0)  # Center of the sphere
radius = 10  # Radius of the sphere
point_a = createPointOnSphere(center, radius, np.pi / 2, 0)
point_b = createPointOnSphere(center, radius, np.pi / 2, 4.87)
num_points = 8  # Number of points to generate

# Generate the points
points_on_plane = generate_points_on_plane(center, radius, point_a, point_b, num_points)
printPoints(points_on_plane)

Copy and paste the following directly into Desmos:
O_0 = (20, 0, 0)
O_1 = (14.14, -14.14, 0)
O_2 = (0, -20, 0)
O_3 = (-14.14, -14.14, 0)
O_4 = (-20, 0, 0)
O_5 = (-14.14, 14.14, 0)
O_6 = (0, 20, 0)
O_7 = (14.14, 14.14, 0)


In [91]:
def pointIsEqualToEither(point, target_one, target_two):
  return np.array_equal(point, target_one) or np.array_equal(point, target_two)

In [92]:
def findShortestPath(path_a, path_b, target_one, target_two):
  for ind in range(0, len(path_a)):
    if pointIsEqualToEither(path_a[ind], target_one, target_two):
      return path_a[:ind + 1]
    if pointIsEqualToEither(path_b[ind], target_one, target_two):
      return path_b[:ind + 1]

In [93]:
# point_b should be the target point
def findShortestFlightPath(target_point, points_on_plane):
  # Get each point's distance from the target point and store as a tuple (point, distanceFromTarget)
  orbital_point_distances_from_target = []
  for point in points_on_plane:
    orbital_point_distances_from_target.append((point, np.linalg.norm(target_point - point)))

  # Sort keys from shortest distance to the target point to the furthest distance
  sorted_distances = sorted(orbital_point_distances_from_target, key=lambda x: x[1])

  # the points themselves
  closest_point_to_target = sorted_distances[0][0]
  second_closest_point_to_target = sorted_distances[1][0]
  third_closest_point_to_target = sorted_distances[2][0]

  # the points' distances from the target
  second_closest_point_distance = sorted_distances[1][1]
  third_closest_point_distance = sorted_distances[2][1]

  path_a = points_on_plane
  # create another list of points where the first point is the same but all others are reversed
  path_b = points_on_plane[:1] + points_on_plane[-1:0:-1]

  # Edge case: target point is almost directly below an orbital point, meaning the second and third closest points are equidistant from the target point
  if abs(second_closest_point_distance - third_closest_point_distance) < 1e-6:
    return findShortestPath(path_a, path_b, second_closest_point_to_target, third_closest_point_to_target)

  return findShortestPath(path_a, path_b, closest_point_to_target, second_closest_point_to_target)

In [94]:
findShortestFlightPath(point_b, points_on_plane)

array([[ 20.  ,   0.  ,   0.  ],
       [ 14.14, -14.14,   0.  ]])